In [17]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [19]:
df_8 = pd.read_excel("tables/Отчет_по_месяцам_8А_2023_2024.xlsx", header=1)
df_9 = pd.read_excel("tables/Отчет_по_месяцам_9А_2024_2025.xlsx", header=1)
df_10 = pd.read_excel("tables/Отчет_по_месяцам_10А_2023_2024.xlsx", header=1)
df_11 = pd.read_excel("tables/Отчет_по_месяцам_11А_2024_2025.xlsx", header=1)
df_8.head(15)

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 244,Unnamed: 245,Unnamed: 246,Unnamed: 247,Unnamed: 248,Unnamed: 249,Unnamed: 250,Unnamed: 251,Unnamed: 252,Unnamed: 253
0,Отчет об успеваемости и посещаемости ученика,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Учебный год: 2023/2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Класс: 8А,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Период: с 01.09.23 по 31.05.24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Ученик 1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Предмет,Сентябрь,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,1,4,5,7,8,11,12,13.0,14,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
def clean_df(df):
    words_to_exclude = ['Год', 'Класс', 'Период', 'год', 'Уважительная', 'Болезнь', 'Неуважительная', 'Опоздание']
    exclude_pattern = '|'.join(words_to_exclude)

    df_cleaned = df[~df.iloc[:, 0].str.contains(exclude_pattern, na=False)]
    df_cleaned = df_cleaned[~df_cleaned.iloc[:, 1].str.contains(exclude_pattern, na=False)]
    df_cleaned = df_cleaned.dropna(how='all')
    df_cleaned = df_cleaned.iloc[1:].reset_index(drop=True)

    return df_cleaned

In [23]:
def create_students_dict(df, rows_per_student):
    student_data = {}
    # Итерируемся по датасету с шагом в 'rows_per_student'
    for i in range(0, len(df), rows_per_student):
        student_df_full = df.iloc[i:i + rows_per_student].copy().reset_index(drop=True)

        # Получаем имя ученика из первой строки
        student_name = student_df_full.iloc[0, 0]  # Предполагаем, что имя ученика в первом столбце

        # Устанавливаем корректные заголовки
        header_subject = student_df_full.iloc[1].fillna('')
        header_date = student_df_full.iloc[2].fillna('')

        new_columns = []
        month = ''
        for j in range(len(header_subject)):
            if header_subject[j] and header_date[j]:
                new_columns.append(f"{header_subject[j]}")
                month = header_subject[j]
            elif header_subject[j]:
                new_columns.append(header_subject[j])
            else:
                str_to_append = f"{month}_{header_date[j]}"
                if str_to_append in new_columns:
                    new_columns.append(f"{month}_{header_date[j]}.1")
                else:
                    new_columns.append(f"{month}_{header_date[j]}")

        student_df_full.columns = new_columns
        student_df_processed = student_df_full.iloc[3:].reset_index(drop=True)
        student_df_processed.rename(columns={student_df_processed.columns[0]: 'Предмет'}, inplace=True)

        # Ищем индекс столбца "Средняя оценка"
        try:
            last_valid_column_index = student_df_processed.columns.get_loc('Средняя оценка') + 1
            # Выбираем все столбцы до "Средняя оценка" включительно
            cleaned_student_df = student_df_processed.iloc[:, :last_valid_column_index].copy()
            student_data[student_name] = cleaned_student_df
        except KeyError:
            print(f"В данных ученика '{student_name}' не найден столбец 'Средняя оценка'.")
            student_data[student_name] = student_df_processed.copy() # Сохраняем весь DataFrame, если столбец не найден

    return student_data

In [407]:
def clean_grade(grade):
    if isinstance(grade, str):
        if '/' in grade:
            try:
                parts = grade.split('/')
                return np.mean([float(x) for x in parts])
            except ValueError:
                return np.nan
        elif grade.strip().lower() in ['у', 'б', 'н']:
            return grade.strip().lower()
    try:
        return float(grade)
    except (ValueError, TypeError):
        return np.nan

def analyze_single_student(student_df):
    metrics = {}
    months_russian = ['Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь', 'Январь', 'Февраль', 'Март', 'Апрель', 'Май']

    # Identify the starting column index for each month
    month_start_indices = {}
    for i, col in enumerate(student_df.columns):
        if col in months_russian:
            month_start_indices[col] = i

    # Define the column ranges for each month
    month_column_ranges = {}
    month_keys = list(month_start_indices.keys())
    for i in range(len(month_keys)):
        month = month_keys[i]
        start_index = month_start_indices[month]
        if i + 1 < len(month_keys):
            end_index = month_start_indices[month_keys[i + 1]]
        else:
            # For the last month, go up to the 'Средняя оценка' column
            if 'Средняя оценка' in student_df.columns:
                end_index = student_df.columns.get_loc('Средняя оценка')
            else:
                end_index = len(student_df.columns)
        month_column_ranges[month] = list(range(start_index, end_index))

    # Collect grades by month
    monthly_grades = {month: [] for month in months_russian}
    non_numeric_monthly = {month: [] for month in months_russian}

    for month, col_indices in month_column_ranges.items():
        month_grade_cols = [student_df.columns[i] for i in col_indices if i < len(student_df.columns)]
        for col in month_grade_cols:
            grades = student_df[col].apply(clean_grade).dropna()
            numeric_grades = pd.to_numeric(grades, errors='coerce').dropna()
            if not numeric_grades.empty:
                monthly_grades[month].extend(numeric_grades.tolist())
            non_numeric = grades[~pd.to_numeric(grades, errors='coerce').notna()]
            if not non_numeric.empty:
                non_numeric_monthly[month].extend(non_numeric.tolist())

    # Расчет среднего балла за месяц
    for month, grades in monthly_grades.items():
        if grades:
            metrics[f'Средний балл за {month}'] = np.mean(grades)
        else:
            metrics[f'Средний балл за {month}'] = np.nan

    # Расчет среднего балла за 3 месяца
    month_per_three = [['Сентябрь', 'Октябрь', 'Ноябрь'], ['Декабрь', 'Январь', 'Февраль'], ['Март', 'Апрель', 'Май']]
    for three_elem in month_per_three:
        m1, m2, m3 = three_elem
        avg1 = metrics.get(f'Средний балл за {m1}')
        avg2 = metrics.get(f'Средний балл за {m2}')
        avg3 = metrics.get(f'Средний балл за {m3}')
        valid_avgs = [avg for avg in [avg1, avg2, avg3] if pd.notna(avg)]
        if valid_avgs:
            metrics[f'Средний балл за 3 месяца ({m1}-{m2}-{m3})'] = np.mean(valid_avgs)
        else:
            metrics[f'Средний балл за 3 месяца ({m1}-{m2}-{m3})'] = np.nan

    # Оценки и статусы за весь год
    all_numeric_grades_year = get_grades_for_months(monthly_grades, months_russian)
    all_non_numeric_year = get_statuses_for_months(non_numeric_monthly, months_russian)

    year_grade_counts = calculate_grade_counts(all_numeric_grades_year)
    for key, value in year_grade_counts.items():
        metrics[f'Общее {key} за год'] = value

    year_status_counts = calculate_status_counts(all_non_numeric_year)
    for key, value in year_status_counts.items():
        metrics[f'Общее {key} за год'] = value

     # Оценки и статусы за полугодия
    first_half_months = ['Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь']
    second_half_months = ['Январь', 'Февраль', 'Март', 'Апрель', 'Май']

    # 1 полугодие
    grades_1_half = get_grades_for_months(monthly_grades, first_half_months)
    statuses_1_half = get_statuses_for_months(non_numeric_monthly, first_half_months)

    grade_counts_1_half = calculate_grade_counts(grades_1_half)
    for key, value in grade_counts_1_half.items():
        metrics[f'Общее {key} за 1 полугодие'] = value
        if len(student_df) > 0:
            metrics[f'Среднее {key} за 1 полугодие (на предмет)'] = value / len(student_df)
        else:
            metrics[f'Среднее {key} за 1 полугодие (на предмет)'] = 0

    status_counts_1_half = calculate_status_counts(statuses_1_half)
    for key, value in status_counts_1_half.items():
        metrics[f'Общее {key} за 1 полугодие'] = value
        if len(student_df) > 0:
            metrics[f'Среднее {key} за 1 полугодие (на предмет)'] = value / len(student_df)
        else:
            metrics[f'Среднее {key} за 1 полугодие (на предмет)'] = 0

    # 2 полугодие
    grades_2_half = get_grades_for_months(monthly_grades, second_half_months)
    statuses_2_half = get_statuses_for_months(non_numeric_monthly, second_half_months)

    grade_counts_2_half = calculate_grade_counts(grades_2_half)
    for key, value in grade_counts_2_half.items():
        metrics[f'Общее {key} за 2 полугодие'] = value
        if len(student_df) > 0:
            metrics[f'Среднее {key} за 2 полугодие (на предмет)'] = value / len(student_df)
        else:
            metrics[f'Среднее {key} за 2 полугодие (на предмет)'] = 0

    status_counts_2_half = calculate_status_counts(statuses_2_half)
    for key, value in status_counts_2_half.items():
        metrics[f'Общее {key} за 2 полугодие'] = value
        if len(student_df) > 0:
            metrics[f'Среднее {key} за 2 полугодие (на предмет)'] = value / len(student_df)
        else:
            metrics[f'Среднее {key} за 2 полугодие (на предмет)'] = 0

    # Расчет среднего количества 5, 4, 3, 2, У, Б, Н в месяц
    num_months_with_grades = len([month for month in months_russian if month in monthly_grades and monthly_grades[month]])
    if num_months_with_grades > 0:
        for grade_value in [5.0, 4.0, 3.0, 2.0]:
            total_count = all_numeric_grades_year.count(grade_value)
            metrics[f'Среднее кол-во {int(grade_value)} в месяц'] = total_count / num_months_with_grades
    else:
        for grade_value in [5.0, 4.0, 3.0, 2.0]:
            metrics[f'Среднее кол-во {int(grade_value)} в месяц'] = np.nan

    num_months_with_status = len([month for month in months_russian if month in non_numeric_monthly and non_numeric_monthly[month]])
    if num_months_with_status > 0:
        for status in ['у', 'б', 'н']:
            total_count = all_non_numeric_year.count(status)
            metrics[f'Среднее кол-во {status.upper()} в месяц'] = total_count / num_months_with_status
    else:
        for status in ['у', 'б', 'н']:
            metrics[f'Среднее кол-во {status.upper()} в месяц'] = np.nan

    # Определение "Успешный" (нет троек за год)
    all_grades_numeric = []
    for month in monthly_grades:
        all_grades_numeric.extend([g for g in monthly_grades[month] if pd.notna(g)])

    if 3.0 in all_grades_numeric:
        metrics['Успешный(нет троек за год)'] = 'Нет'
    else:
        metrics['Успешный(нет троек за год)'] = 'Да'

    return metrics

# Предполагается, что у тебя уже есть функция create_students_dict,
# которая создает словарь student_dict

# Пример использования:
# metrics_stud1 = analyze_single_student(stud1)
# print(metrics_stud1)

# Чтобы применить ко всем ученикам:
# all_students_metrics = []
# for student_name, student_df in student_dict.items():
#     metrics = analyze_single_student(student_df)
#     metrics['Ученик'] = student_name
#     all_students_metrics.append(metrics)

# final_metrics_df = pd.DataFrame(all_students_metrics)
# final_metrics_df

In [53]:
class StudentAnalyzer:
    def __init__(self, student_df):
        self.student_df = student_df
        self.months = ['Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь', 'Январь', 'Февраль', 'Март', 'Апрель', 'Май']
        self.monthly_grades = {month: [] for month in self.months}
        self.non_numeric_monthly = {month: [] for month in self.months}
        self.metrics = {}
        self._extract_month_data()

    @staticmethod
    def clean_grade(grade):
        if isinstance(grade, str):
            if '/' in grade:
                try:
                    return np.mean([float(x) for x in grade.split('/')])
                except ValueError:
                    return np.nan
            elif grade.strip().lower() in ['у', 'б', 'н']:
                return grade.strip().lower()
        try:
            return float(grade)
        except (ValueError, TypeError):
            return np.nan

    @staticmethod
    def calculate_grade_counts(grades_list):
        return {f'кол-во {int(g)}': grades_list.count(g) for g in [5.0, 4.0, 3.0, 2.0]}

    @staticmethod
    def calculate_status_counts(statuses_list):
        return {f'кол-во {s.upper()}': statuses_list.count(s) for s in ['у', 'б', 'н']}

    def _extract_month_data(self):
        month_start_indices = {col: i for i, col in enumerate(self.student_df.columns) if col in self.months}
        month_keys = list(month_start_indices.keys())

        month_column_ranges = {}
        for i, month in enumerate(month_keys):
            start = month_start_indices[month]
            if i + 1 < len(month_keys):
                end = month_start_indices[month_keys[i + 1]]
            elif 'Средняя оценка' in self.student_df.columns:
                end = self.student_df.columns.get_loc('Средняя оценка')
            else:
                end = len(self.student_df.columns)
            month_column_ranges[month] = list(range(start, end))

        for month, indices in month_column_ranges.items():
            for col in [self.student_df.columns[i] for i in indices]:
                cleaned = self.student_df[col].apply(self.clean_grade).dropna()
                numeric = pd.to_numeric(cleaned, errors='coerce').dropna()
                if not numeric.empty:
                    self.monthly_grades[month].extend(numeric.tolist())
                non_numeric = cleaned[~pd.to_numeric(cleaned, errors='coerce').notna()]
                if not non_numeric.empty:
                    self.non_numeric_monthly[month].extend(non_numeric.tolist())

    def analyze(self):
        self._calculate_monthly_averages()
        self._calculate_three_month_averages()
        self._calculate_year_and_semesters_stats()
        self._calculate_monthly_averages_counts()
        self._determine_success()
        return self.metrics

    def _calculate_monthly_averages(self):
        for month in self.months:
            grades = self.monthly_grades[month]
            self.metrics[f'Средний балл за {month}'] = np.mean(grades) if grades else np.nan

    def _calculate_three_month_averages(self):
        blocks = [['Сентябрь', 'Октябрь', 'Ноябрь'], ['Декабрь', 'Январь', 'Февраль'], ['Март', 'Апрель', 'Май']]
        for block in blocks:
            avgs = [self.metrics[f'Средний балл за {m}'] for m in block if pd.notna(self.metrics[f'Средний балл за {m}'])]
            self.metrics[f'Средний балл за 3 месяца ({block[0]}-{block[1]}-{block[2]})'] = np.mean(avgs) if avgs else np.nan

    def _calculate_year_and_semesters_stats(self):
        all_grades = self._get_combined_grades(self.months)
        all_statuses = self._get_combined_statuses(self.months)

        for key, val in self.calculate_grade_counts(all_grades).items():
            self.metrics[f'Общее {key} за год'] = val
        for key, val in self.calculate_status_counts(all_statuses).items():
            self.metrics[f'Общее {key} за год'] = val

        self._semester_stats('1 полугодие', ['Сентябрь', 'Октябрь', 'Ноябрь', 'Декабрь'])
        self._semester_stats('2 полугодие', ['Январь', 'Февраль', 'Март', 'Апрель', 'Май'])

    def _semester_stats(self, label, months):
        grades = self._get_combined_grades(months)
        statuses = self._get_combined_statuses(months)
        row_count = len(self.student_df)

        for key, val in self.calculate_grade_counts(grades).items():
            self.metrics[f'Общее {key} за {label}'] = val
            self.metrics[f'Среднее {key} за {label} (на предмет)'] = val / row_count if row_count > 0 else 0
        for key, val in self.calculate_status_counts(statuses).items():
            self.metrics[f'Общее {key} за {label}'] = val
            self.metrics[f'Среднее {key} за {label} (на предмет)'] = val / row_count if row_count > 0 else 0

    def _calculate_monthly_averages_counts(self):
        valid_months_grades = [m for m in self.months if self.monthly_grades[m]]
        valid_months_statuses = [m for m in self.months if self.non_numeric_monthly[m]]

        for grade in [5.0, 4.0, 3.0, 2.0]:
            total = sum([self.monthly_grades[m].count(grade) for m in valid_months_grades])
            self.metrics[f'Среднее кол-во {int(grade)} в месяц'] = total / len(valid_months_grades) if valid_months_grades else np.nan

        for status in ['у', 'б', 'н']:
            total = sum([self.non_numeric_monthly[m].count(status) for m in valid_months_statuses])
            self.metrics[f'Среднее кол-во {status.upper()} в месяц'] = total / len(valid_months_statuses) if valid_months_statuses else np.nan

    def _determine_success(self):
        """
        Успешный студент — если по каждому предмету (строке) средний балл >= 3.5
        """
        all_columns = []
        for month in self.months:
            all_columns.extend([col for col in self.student_df.columns if col.startswith(month)])
        
        for _, row in self.student_df.iterrows():
            grades = []
            for col in all_columns:
                if col in row:
                    val = StudentAnalyzer.clean_grade(row[col])
                    if isinstance(val, float) and pd.notna(val):
                        grades.append(val)
            if grades:
                avg = np.mean(grades)
                if avg < 3.5:
                    self.metrics['Успешный (нет предмета со ср. баллом < 3.5)'] = 0
                    return
        self.metrics['Успешный (нет предмета со ср. баллом < 3.5)'] = 1

    def _get_combined_grades(self, months):
        grades = []
        for m in months:
            grades.extend(self.monthly_grades[m])
        return [g for g in grades if pd.notna(g)]

    def _get_combined_statuses(self, months):
        statuses = []
        for m in months:
            statuses.extend(self.non_numeric_monthly[m])
        return statuses


In [133]:
def create_dataset_for_clast(student_dict):
    all_students_metrics = []
    for student_name, student_df in student_dict.items():
        analyzer = StudentAnalyzer(student_df)
        metrics = analyzer.analyze()
        metrics['Ученик'] = student_name
        all_students_metrics.append(metrics)
    final_metrics_df = pd.DataFrame(all_students_metrics)
    
    column_list = final_metrics_df.columns.tolist()
    last_column = column_list[-1]
    other_columns = column_list[:-1]
    new_column_order = [last_column] + other_columns
    final_metrics_df = final_metrics_df[new_column_order]
    
    return final_metrics_df

In [135]:
df_8_clean = clean_df(df_8.copy())
df_9_clean = clean_df(df_9.copy())
df_10_clean = clean_df(df_10.copy())
df_11_clean = clean_df(df_11.copy())
student_dict_8 = create_students_dict(df_8_clean, 24)
student_dict_9 = create_students_dict(df_9_clean, 20)
student_dict_10 = create_students_dict(df_10_clean, 24)
student_dict_11 = create_students_dict(df_11_clean, 22)

In [153]:
final_metrics_df_8 = create_dataset_for_clast(student_dict_8)
final_metrics_df_9 = create_dataset_for_clast(student_dict_9)
final_metrics_df_10 = create_dataset_for_clast(student_dict_10)
final_metrics_df_11 = create_dataset_for_clast(student_dict_11)

In [154]:
try:
    final_metrics_df_9 = final_metrics_df_9.drop(columns=['Средний балл за Май'])
    final_metrics_df_11 = final_metrics_df_11.drop(columns=['Средний балл за Май'])
except:
    pass

,Ученик,Средний балл за Сентябрь,Средний балл за Октябрь,Средний балл за Ноябрь,Средний балл за Декабрь,Средний балл за Январь,Средний балл за Февраль,Средний балл за Март,Средний балл за Апрель,Средний балл за 3 месяца (Сентябрь-Октябрь-Ноябрь),...,Общее кол-во Н за 2 полугодие,Среднее кол-во Н за 2 полугодие (на предмет),Среднее кол-во 5 в месяц,Среднее кол-во 4 в месяц,Среднее кол-во 3 в месяц,Среднее кол-во 2 в месяц,Среднее кол-во У в месяц,Среднее кол-во Б в месяц,Среднее кол-во Н в месяц,Успешный (нет предмета со ср. баллом < 3.5)
0,Ученик 1,4.294118,4.204082,4.641026,4.454545,4.555556,4.516129,4.304348,4.208333,4.379742,...,0,0.000000,15.875,12.125,1.875,0.750,5.800000,3.400000,0.000000,1
1,Ученица 2,4.777778,4.804348,4.956522,4.939394,4.928571,5.000000,4.982759,4.769231,4.846216,...,3,0.157895,26.125,1.750,0.500,0.125,18.666667,0.000000,1.333333,1
2,Ученица 3,4.166667,3.655556,3.797297,4.052632,3.968750,4.068966,3.857143,3.681818,3.873173,...,3,0.157895,8.500,10.250,7.375,2.125,4.428571,4.857143,1.142857,0
3,Ученик 4,4.285714,4.648936,4.878378,4.714286,4.827586,4.750000,5.000000,4.666667,4.604343,...,3,0.157895,20.750,4.375,0.500,0.250,20.400000,0.000000,0.800000,1
4,Ученица 5,4.257143,4.250000,4.346154,4.187500,4.405405,4.000000,4.055556,4.181818,4.284432,...,2,0.105263,13.000,13.000,4.375,0.750,6.571429,0.000000,0.571429,1
5,Ученица 6,4.193548,4.288889,4.527778,4.294118,4.314286,4.310345,4.357143,4.000000,4.336738,...,1,0.052632,13.500,14.000,2.875,0.500,6.500000,0.000000,0.500000,1
6,Ученик 7,4.193548,3.906250,4.231707,4.181818,4.151515,4.238095,4.192308,3.791667,4.110502,...,0,0.000000,10.625,13.250,5.875,0.500,4.500000,3.833333,0.166667,0
7,Ученик 8,3.862069,3.635417,3.404762,3.625000,3.657143,3.500000,3.666667,3.750000,3.634083,...,3,0.157895,6.625,8.500,11.375,3.250,5.750000,0.000000,1.750000,0
8,Ученик 9,3.689655,3.540816,3.947368,3.764706,4.037037,3.481481,3.791667,3.357143,3.725947,...,4,0.210526,4.750,9.000,6.750,2.500,2.250000,15.000000,1.125000,0
9,Ученик 10,3.684211,3.476744,3.272727,3.785714,4.000000,3.807692,3.900000,3.166667,3.477894,...,2,0.105263,5.125,7.875,8.375,3.125,7.125000,15.000000,1.000000,0


In [157]:
final_metrics_df_8.to_csv('datasets/8A_для_кластеризации.csv',index=False, encoding="utf-8-sig")
final_metrics_df_9.to_csv('datasets/9A_для_кластеризации.csv',index=False, encoding="utf-8-sig")
final_metrics_df_10.to_csv('datasets/10A_для_кластеризации.csv',index=False, encoding="utf-8-sig")
final_metrics_df_11.to_csv('datasets/11A_для_кластеризации.csv',index=False, encoding="utf-8-sig")